
# Folium Data Analysis and Visualization Exercise


**Dataset**: You will be using the [Chicago Crime Dataset (2001 - Present)](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2).

**Objectives**:
1. Load and inspect the dataset.
2. Set up a map and add crime markers based on different crime types.
3. Marker clusters to visualize high-density crime areas.
4. Create toggle layers for specific types of crimes.


### Importing Libraries

In [90]:
import numpy as np
import pandas as pd
import folium

### Importing the dataset we will use to get the criminal records

In [ ]:
url = "https://data.cityofchicago.org/resource/crimes.csv"

df_incidents = pd.read_csv(url)

df_incidents.head()

#### It is a big data set as I will show so will only use a small part of it to visuzalize it more accurately

In [ ]:
df_incidents.shape

(1000, 22)

In [ ]:
df_incidents.isnull().sum()

id                       0
case_number              0
date                     0
block                    0
iucr                     0
primary_type             0
description              0
location_description     3
arrest                   0
domestic                 0
beat                     0
district                 0
ward                     0
community_area           0
fbi_code                 0
x_coordinate            14
y_coordinate            14
year                     0
updated_on               0
latitude                14
longitude               14
location                14
dtype: int64

In [ ]:
df_incidents.dropna(inplace=True)
df_incidents.isnull().sum()

id                      0
case_number             0
date                    0
block                   0
iucr                    0
primary_type            0
description             0
location_description    0
arrest                  0
domestic                0
beat                    0
district                0
ward                    0
community_area          0
fbi_code                0
x_coordinate            0
y_coordinate            0
year                    0
updated_on              0
latitude                0
longitude               0
location                0
dtype: int64

In [ ]:
df_incidents = df_incidents.head(200)
print("Successful! you have selected the first 200 rows of the dataset")

Successful! you have selected the first 200 rows of the dataset


### Setting up the map and adding the crime markers based on different crime types

#### Setting up the map focusing on Chicago

In [ ]:
chicago_map = folium.Map(location=[41.881832,-87.623177], zoom_start= 10)
chicago_map

#### The function that adds markers to the map

In [ ]:
df_incidents['primary_type'].unique()

array(['CRIMINAL DAMAGE', 'ASSAULT', 'THEFT', 'BURGLARY', 'ROBBERY',
       'DECEPTIVE PRACTICE', 'WEAPONS VIOLATION', 'BATTERY',
       'OTHER OFFENSE', 'NARCOTICS', 'PUBLIC PEACE VIOLATION',
       'MOTOR VEHICLE THEFT', 'SEX OFFENSE', 'CRIMINAL TRESPASS',
       'PROSTITUTION', 'INTERFERENCE WITH PUBLIC OFFICER',
       'LIQUOR LAW VIOLATION'], dtype=object)

#### Defining each crime and assigning a color to each of them

In [ ]:
# Define colors for each crime category
crime_colors = {
    'CRIMINAL DAMAGE': 'red',
    'ASSAULT': 'orange',
    'DECEPTIVE PRACTICE': 'blue',
    'ROBBERY': 'green',
    'THEFT': 'purple',
    'BURGLARY': 'brown',
    'WEAPONS VIOLATION': 'black',
    'BATTERY': 'yellow',
    'OTHER OFFENSE': 'gray',
    'NARCOTICS': 'pink',
    'PUBLIC PEACE VIOLATION': 'cyan',
    'MOTOR VEHICLE THEFT': 'darkblue',
    'SEX OFFENSE': 'violet',
    'CRIMINAL TRESPASS': 'lightgreen',
    'PROSTITUTION': 'darkred',
    'INTERFERENCE WITH PUBLIC OFFICER': 'lightblue',
    'LIQUOR LAW VIOLATION': 'gold',
    'OFFENSE INVOLVING CHILDREN': 'lightpink',
    'ARSON': 'firebrick',
    'STALKING': 'darkviolet',
    'CRIMINAL SEXUAL ASSAULT': 'darkgreen'
}

#### For loop to add markers and changing it's color to match each crime

In [ ]:
incidents = folium.FeatureGroup()

# Adding CircleMarkers with specific colors for each crime
for lat, lng, category in zip(df_incidents.latitude, df_incidents.longitude, df_incidents.primary_type):
    color = crime_colors.get(category, 'black')  # If the crime in our selection wasn't found return to 'black'
    incidents.add_child(folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        icon = None,
    ))

# Add markers with popup labels for crime categories
for lat, lng, label, category in zip(df_incidents.latitude, df_incidents.longitude, df_incidents.primary_type, df_incidents.primary_type):
    color = crime_colors.get(category, 'black')
    folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color=color, icon_color='white', icon='exclamation-circle', prefix='fa') 
    ).add_to(chicago_map)

# Add the incidents feature group to the map
chicago_map.add_child(incidents)

C:\Users\maraw\AppData\Local\Temp\ipykernel_183040\794583060.py:22: UserWarning: color argument of Icon should be one of: {'cadetblue', 'gray', 'black', 'orange', 'darkred', 'white', 'beige', 'lightgray', 'darkpurple', 'green', 'darkblue', 'lightblue', 'red', 'darkgreen', 'purple', 'lightred', 'lightgreen', 'blue', 'pink'}.
  icon=folium.Icon(color=color, icon_color='white', icon='exclamation-circle', prefix='fa')


### Using the Clustering method

#### Creating a new map to get rid of the markers

In [ ]:
chicago_map = folium.Map(location=[41.881832,-87.623177], zoom_start= 10)

#### Adding the clusters to show the density of the criminal records in each area

In [ ]:
from folium import plugins
incidents = plugins.MarkerCluster().add_to(chicago_map)

labels = list(df_incidents.primary_type)

for lat, lng, label in zip(df_incidents.latitude, df_incidents.longitude, labels):
    folium.Marker(
        location = [lat, lng],
        icon = folium.Icon(color='blue', icon='exclamation-circle', prefix='fa'),
        popup = label
    ).add_to(incidents)
    
chicago_map.add_child(incidents)

### Heatmap for each crime

In [ ]:
chicago_map = folium.Map(location=[41.881832,-87.623177], zoom_start= 10)

In [ ]:
# Extract crime data from our dataset
locations = list(zip(df_incidents['latitude'], df_incidents['longitude'])) 

# Creating a heatmap using the latitudes and longitudes pairs we obtained 
heat_data = [[lat, lng] for lat, lng in locations]  

plugins.HeatMap(heat_data).add_to(chicago_map)

chicago_map


## Bonus:  Adding toggle layers to filter each crime and only display the selected one

In [ ]:
chicago_map = folium.Map(location=[41.881832,-87.623177], zoom_start= 10)

#### The map is set to be empty in the beginning with the layers legend on the top right to choose the specific crime you want to see

In [ ]:
# Creating a crime category for each crime category
crime_categories = {}

# Looping through the crime_colors dictionary to create crime categories
for category, color in crime_colors.items():
    crime_category = folium.FeatureGroup(name=category, show=False)  # Creating a new crime category for each category # Show = False to start with the map being empty
    
    # Filtering the dataset for each crime category
    category_data = df_incidents[df_incidents['primary_type'] == category]
    
    # Adding CircleMarkers for each crime
    for lat, lng in zip(category_data.latitude, category_data.longitude):
        folium.features.CircleMarker(
            [lat, lng],
            radius=10,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=category  
        ).add_to(crime_category)  # Adding CircleMarker to the corresponding crime category

    # Adding the crime category to the map after it's populated with CircleMarkers
    crime_categories[category] = crime_category

# Adding all crime categories to the map
for category in crime_categories.values():
    category.add_to(chicago_map)

# Adding LayerControl to allow toggling crime categories
folium.LayerControl().add_to(chicago_map)

# Showing the map
chicago_map
